In [29]:
%run load_data.py

In [34]:
from query import *

In [2]:
ss = query(chaindata, chainschema, fragments, is_ss, "ss", part=None)

In [3]:
ds = query(chaindata, chainschema, fragments, is_ds, "ss", part=None)

In [4]:
ss_ph = query(chaindata, chainschema, fragments[ss], contact_parts, "interface_protein", part="ph")

In [5]:
ds_ph = query(chaindata, chainschema, fragments[ds], contact_parts, "interface_protein", part="ph")

In [ ]:
100*sum(ds)/len(fragments)

In [ ]:
100*sum(ds_ph)/sum(ds)

In [ ]:
ss_base = query(chaindata, chainschema, fragments[ss], contact_parts, "interface_protein", part="base")

In [ ]:
ss_sug = query(chaindata, chainschema, fragments[ss], contact_parts, "interface_protein", part="sug")

In [ ]:
counts={}
for part in ["ph","sug", "base"]:
    for statename, state in zip(["ss","ds"],[ss, ds]):
        print(part, statename)
        counts[statename, part] = np.array(query(chaindata, chainschema, fragments[state], contact_parts, "interface_protein", part))

In [ ]:
sum(counts["ss","base"])

In [ ]:
ss_contact = sum([sum(counts["ss",i]) for i in ["sug", "ph", "base"]])

In [ ]:
ds_contact = sum([sum(counts["ds",i]) for i in ["sug", "ph", "base"]])

In [ ]:
def sum_bool(counts, statename):
    mask = counts[statename,"sug"] | counts[statename,"ph"] | counts[statename,"base"]
    print(len(mask))
    return sum(mask)

In [ ]:
ds_all=sum_bool(counts, "ds")
ss_all=sum_bool(counts, "ss")

In [ ]:
ds_all

In [ ]:
ss_all

In [ ]:
100*sum(counts["ss","base"])/ss_all

In [ ]:
100*sum(counts["ds","base"])/ds_all

In [ ]:
100*sum(counts["ss","ph"])/ss_all

In [ ]:
100*sum(counts["ds","ph"])/ds_all

In [ ]:
100*sum(counts["ss","sug"])/ss_all

In [ ]:
100*sum(counts["ds","sug"])/ds_all

In [ ]:
sum(counts["ss","base"])

In [ ]:
ss_all - sum(counts["ss","base"])

In [ ]:
sum(counts["ds","base"])

In [ ]:
ds_all - sum(counts["ds","base"])

In [ ]:
from scipy.stats import fisher_exact
a=[[22043, 7248],[68126, 48909]]
fisher_exact(a)

In [ ]:
p = "ph"
a = sum(counts["ss",p])
b = ss_all - sum(counts["ss",p])
c = sum(counts["ds",p])
d = ds_all - sum(counts["ds",p])
table = [[a,b],[c,d]]
fisher_exact(table)

In [ ]:
stats={ "ph":[100*sum(counts["ss","ph"])/ss_all, 100*sum(counts["ds","ph"])/ds_all],
       "sug":[100*sum(counts["ss","sug"])/ss_all, 100*sum(counts["ds","sug"])/ds_all],
       "base": [100*sum(counts["ss","base"])/ss_all, 100*sum(counts["ds","base"])/ds_all] }

In [ ]:
stats

In [ ]:
frag20=fragments[:10]
frag20

In [ ]:
import pandas as pd
pd.DataFrame.from_records(frag20[""])

In [ ]:
mask1=(fragments["seq"]== b'ACU')
mask2=(fragments["structure"]==b'1OOA')
frag=fragments[mask1&mask2][-1]

In [ ]:
r=query_one_frag(chaindata, chainschema, frag, "interface_protein", "ph")
contact_parts(r)

# Question 2
For single-stranded fragments where the middle residue contacts with the phosphate,
 how many stacking interactions does it make with neighboring bases?

In [8]:
# Select single-stranded fragments
ss = query(chaindata, chainschema, fragments, is_ss, "ss", part=None)
frag_ss = fragments[ss]

# mask (True or False for every fragment in frag_ss) on frags with central nucl in contact with prot via ph
ss_ph = np.array(query(chaindata, chainschema, frag_ss, contact_parts, "interface_protein", part="ph"))

In [9]:
frag_ss = fragments[ss]
# mask on frag_ss with central base stacking with base n-1
ss_stackprev = query(chaindata, chainschema, frag_ss, contact_parts, "stacking", pos="n-1")

In [10]:
# mask on frag_ss with central base stacking with base n+1
ss_stacknext = query(chaindata, chainschema, frag_ss, contact_parts, "stacking", pos="n+1")

In [11]:
a = sum(ss_stacknext)
b = sum(ss_stackprev)
print(a, b)

7599 7547


In [12]:
ss_stack = np.array(ss_stackprev,int)+np.array(ss_stacknext,int)

In [13]:
#number of stack for each frag_ss that bind via ph
stack_ph = ss_stack[ss_ph]

#number of stack for each frag_ss that does not bind via ph
stack_noph = ss_stack[~ss_ph]

In [21]:
import numpy as np
data1 = np.unique(stack_ph,return_counts=True)
a1 = data1[1][0] + data1[1][1]
b1 = data1[1][2]

In [22]:
data2 = np.unique(stack_noph,return_counts=True)
a2 = data2[1][0] + data2[1][1]
b2 = data2[1][2]

In [19]:
len(ss_ph)

45960

In [26]:
stack_ph

array([0, 0, 0, ..., 0, 0, 1])

In [23]:
from scipy.stats import fisher_exact
table = [[a1,b1],  [a2,b2]]
fisher_exact(table)

(1.3071722310014893, 1.7252797645237667e-14)

In [62]:
# percent of [frag_ss binding by phosphate] that has 2 stackings
print(np.sum(stack_ph==2)/len(stack_ph)*100)
# percent of [frag_ss binding by non-phosphate] that has 2 stackings
print(np.sum(stack_noph==2)/len(stack_noph)*100)

6.89806503092
8.82986360373


In [64]:
# percent of frag_ss with 2 stackings
(ss_stack==2).sum()/len(frag_ss)*100

7.7763272410791995

In [66]:
# percent of binding frag_ss that binds via ph
sum(ss_ph)/len(frag_ss)*100

54.536553524804177

In [37]:
%run query
ss_ph_3p5 = query(chaindata, chainschema, fragments[ss], set_contact_parts(3.5), "interface_protein", part="ph")

In [40]:
sum(ss_ph_3p5)

15365

In [41]:
sum(ss_ph)

25065

In [ ]:
# percent of [frag_ss with <3.5 phosphate] that has 2 stackings
print(np.sum(stack_ph==2)/len(stack_ph)*100)
# percent of [frag_ss with >3.5 phosphate] that has 2 stackings
print(np.sum(stack_noph==2)/len(stack_noph)*100)